In [1]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import BallTree
import faiss

In [2]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from Bio import SeqIO

sequence_file = '/mnt/home/thamamsy/ceph/scrap/cafa/embeddings/Train/train_sequences.fasta'
record_ids = []
record_seqs = []

with open(sequence_file) as handle:
    for record in SeqIO.parse(handle, "fasta"):
        record_ids.append(record.id)
        record_seqs.append(str(record.seq))
        
lengths = [len(record_seqs[i]) for i in range(len(record_seqs))]
record_df = pd.DataFrame({'Id': record_ids, 'Seq': record_seqs, 'Length': lengths})
record_df_below_seq = record_df[record_df['Length'] < 2500]
sequences = list(record_df_below_seq['Seq'].values)

In [3]:
len(record_ids)

142246

In [4]:
#record_df_below_seq

In [5]:
len(sequences)

140478

In [5]:
#record_df

In [3]:
train_terms = pd.read_csv("/mnt/home/thamamsy/ceph/scrap/cafa/embeddings/Train/train_terms.tsv", sep = '\t')

In [4]:
train_embeddings = np.load('../cafa/tm_vec_train.npy')
#test_embeddings =
# Now lets convert embeddings numpy array(train_embeddings) into pandas dataframe.
column_num = train_embeddings.shape[1]
train_df = pd.DataFrame(train_embeddings, columns = ["Column_" + str(i) for i in range(1, column_num+1)], 
                        index = record_df_below_seq.Id)
print(train_df.shape)

(140478, 512)


In [5]:
df = train_df.sample(frac=1)
train_df1 = df.iloc[:len(train_df) * 8//10]
test_df1 = df.iloc[len(train_df) * 8//10:]
# Convert the DataFrame to a NumPy array
data_array = train_df1.to_numpy(dtype='float32')
# Build the BallTree index
index = faiss.IndexFlatL2(data_array.shape[1])  # L2 distance is used for the BallTree
index.add(data_array)

# Convert the test DataFrame to a NumPy array
test_array = test_df1.to_numpy(dtype='float32')

# Perform nearest neighbor search with the BallTree index
k = 1  # Number of nearest neighbors to find
distances, indices = index.search(test_array, k)

# distances: array of shape (num_queries, k) containing the distances to the k nearest neighbors
# indices: array of shape (num_queries, k) containing the indices of the k nearest neighbors in the original data

# Now we have the nearest neighbors' indices and distances for each test sample.
# access the corresponding data points in the original DataFrame as follows:
nearest_neighbors_df = train_df1.iloc[indices[:, 0]]

In [75]:
sparse_matrix1

<28096x22615 sparse matrix of type '<class 'numpy.int64'>'
	with 1111268 stored elements in Compressed Sparse Row format>

In [78]:
# train_terms -- nearest neighbors for testing
train_terms = pd.read_csv("/mnt/home/thamamsy/ceph/scrap/cafa/embeddings/Train/train_terms.tsv", sep = '\t')
selected_ids = nearest_neighbors_df.index
train_terms.set_index('EntryID', inplace=True)
nearest_neighbors = train_terms.loc[selected_ids]
nearest_neighbors = nearest_neighbors.drop('aspect', axis=1)
# Use pivot_table to create the new DataFrame
new_nearest_neighbors = nearest_neighbors.pivot_table(index='Id', columns='term', aggfunc=lambda x: 1, fill_value=0)

#test_df1
train_terms = pd.read_csv("/mnt/home/thamamsy/ceph/scrap/cafa/embeddings/Train/train_terms.tsv", sep = '\t')
selected_ids = test_df1.index
train_terms.set_index('EntryID', inplace=True)
test_df1_labels = train_terms.loc[selected_ids]
test_df1_labels = test_df1_labels.drop('aspect', axis=1)
# Use pivot_table to create the new DataFrame
new_test_df1_labels = test_df1_labels.pivot_table(index='Id', columns='term', aggfunc=lambda x: 1, fill_value=0)

test_df1_Go = pd.DataFrame(new_test_df1_labels, index=test_df1.index)
nearest_neighbors_Go = pd.DataFrame(new_nearest_neighbors, index=nearest_neighbors_df.index)

#AUPR for multi-label output
nearest_neighbors_Go_new_index = nearest_neighbors_Go.reset_index(drop=True)
nearest_neighbors_Go_new_index.index = nearest_neighbors_Go_new_index.index + 1
#test_df1_Go
test_df1_Go_new_index = test_df1_Go.reset_index(drop=True)
test_df1_Go_new_index.index = test_df1_Go_new_index.index + 1

from scipy.sparse import csr_matrix
from sklearn.metrics import average_precision_score

sparse_matrix1 = csr_matrix(nearest_neighbors_Go_new_index)
sparse_matrix2 = csr_matrix(test_df1_Go_new_index)

# Check the shapes of the matrices
shape1 = sparse_matrix1.shape
shape2 = sparse_matrix2.shape

# If the shapes are different, resize the matrices to the same shape
if similarities != shape2:
    # Choose the common shape that both matrices will be resized to
    common_shape = (max(similarities[0], shape2[0]), max(similarities[1], shape2[1]))

    # Resize the matrices to the common shape
    similarities.resize(common_shape)
    sparse_matrix2.resize(common_shape)

# Calculate the similarity scores using the k-NN algorithm and assume they are stored in the "similarities" variable
similarities = np.exp(-distances)

# Flatten the matrices and similarity scores for AUPR calculation
flattened_matrix1 = sparse_matrix1.toarray().flatten()
flattened_matrix2 = sparse_matrix2.toarray().flatten()
flattened_similarities = similarities.flatten()

# Calculate AUPR using similarity scores and matrix2 as ground truth
aupr_score = average_precision_score(flattened_matrix2, flattened_similarities)
print("AUPR Score:", aupr_score)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [69]:
print("Shape of flattened_matrix2:", flattened_matrix2.shape)
print("Shape of flattened_similarities:", flattened_similarities.shape)


Shape of flattened_matrix2: (635391040,)
Shape of flattened_similarities: (28096,)


In [63]:
from scipy.sparse import csr_matrix
from sklearn.metrics import average_precision_score

sparse_matrix1 = csr_matrix(nearest_neighbors_Go_new_index)
sparse_matrix2 = csr_matrix(test_df1_Go_new_index)

# Check the shapes of the matrices
shape1 = sparse_matrix1.shape
shape2 = sparse_matrix2.shape

# If the shapes are different, resize the matrices to the same shape
if shape1 != shape2:
    # Choose the common shape that both matrices will be resized to
    common_shape = (max(shape1[0], shape2[0]), max(shape1[1], shape2[1]))

    # Resize the matrices to the common shape
    sparse_matrix1.resize(common_shape)
    sparse_matrix2.resize(common_shape)

# Calculate the similarity scores using the k-NN algorithm and assume they are stored in the "similarities" variable
similarities = np.exp(-distances)

# Flatten the matrices and similarity scores for AUPR calculation
flattened_matrix1 = sparse_matrix1.toarray().flatten()
flattened_matrix2 = sparse_matrix2.toarray().flatten()
flattened_similarities = similarities.flatten()

# Calculate AUPR using similarity scores and matrix2 as ground truth
aupr_score = average_precision_score(flattened_matrix2, flattened_matrix2)
print("AUPR Score:", aupr_score)


AUPR Score: 0.0017583177259355017
